<a href="https://colab.research.google.com/github/ph20d204/Instrument_Automation/blob/main/LifeTimeMeasurement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# !pip install pulsestreamer
import numpy as np
import inspect,dis
import sys
from time import *
import pulsestreamer
import pprint
from pulsestreamer import PulseStreamer,findPulseStreamers,OutputState,TriggerStart,Sequence

In [ ]:
IPaddress = findPulseStreamers(search_serial='')[0][0]
pulser = PulseStreamer(IPaddress)

In [43]:
# all in nanosecond range
delay1=50; delay2=20
laser_on=1000; Tmax=3000
read_on=150
steps=5; samples=1000; averages=10
laser_port=0; readout_port=1

t_vec=np.linspace(0,Tmax,num=steps)

In [44]:
def lifetime(delay1,delay2,laser_on,t_vec,read_on,averages,samples,*ports):  # all in nanosecond range

  ## Checking the initial condition of Pulse Streamer
  # if pulser.hasSequence()!=0:
  #   print('Pulse Streamer has a Sequence. Do you want to proceed?\n')
  #   permission = int(input('Press 0/1 : \n'))
  # if permission==0:
  #   pass

  ## Producing the Patterns
  laser_block = [(delay2,0),(laser_on,1),(delay2+t_vec[-1],0)]
  laser_pattern = [(delay1,0)]; signal_pattern = [(delay1,0)]

  for t in range(t_vec.shape[0]-1):
    if t_vec.shape[0] > t_vec[-1]/read_on:
      print(f'Please reduce the step numbers. Maximum allowed steps: {t_vec[-1]/read_on}')
      break
    laser_pattern += laser_block
    signal_pattern += [((2*delay2+laser_on),0),(read_on,1),(delay2+t_vec[t],0),(read_on,1),(t_vec[-1]-t_vec[t]-delay2-2*read_on,0)]

  if t_vec.shape[0] > t_vec[-1]/read_on:
    pass
  laser_pattern +=[(delay1,0)]; signal_pattern +=[(delay1,0)]

  # ## Producing the Sequences
  # reSet = pulser.reset()
  # if reSet==0:
  #   print('Pulse Streamer has been reset.\n')
  # seq = pulser.createSequence()
  # laser_port=ports[0]; ref_port=ports[1]; read_port=ports[2]
  # seq.setDigital(laser_port,laser_pattern) ; seq.setDigital(ref_port,ref_pattern) ; seq.setDigital(read_port,read_pattern)
  # print(f'Plot of the Sequence without averaging:\n {seq.plot()}\n')
  # seq*=averages

  # ## Checking the Conditions
  # if pulser.hasSequence()!=0:
  #   print('Pulse Streamer has no Sequence uploaded!')
  #   pass
  # if seq.isEmpty()==0:
  #   print('Sequence to Pulse Streamer is empty!')
  #   pass
  # if seq.getDuration()%8==0:
  #   print("Sequence duration is not multiple of 8ns!\n Please check by 'seq.get_pad()' method.")
  #   pass

  # ## Performing the Streaming
  # pulser.stream(seq,n_runs=samples,final=([],0,0))
  # if pulser.hasFinished()==1:
  #   print('Streaming has been finished.\n')
  # reSet = pulser.reset()
  # if reSet==0:
  #   print('Pulse Streamer has been reset after Streaming.\n')

  return laser_pattern, signal_pattern

In [45]:
# ## Choosing the Devices
# use_pulser=input('Enter your specific Pulse Streamer: ')
# use_nidaq=input('Enter your specific NIDAQmx: ')

lifetime(delay1,delay2,laser_on,t_vec,read_on,averages,samples,laser_port,readout_port)

([(50, 0),
  (20, 0),
  (1000, 1),
  (3020.0, 0),
  (20, 0),
  (1000, 1),
  (3020.0, 0),
  (20, 0),
  (1000, 1),
  (3020.0, 0),
  (20, 0),
  (1000, 1),
  (3020.0, 0),
  (50, 0)],
 [(50, 0),
  (1040, 0),
  (150, 1),
  (20.0, 0),
  (150, 1),
  (2680.0, 0),
  (1040, 0),
  (150, 1),
  (770.0, 0),
  (150, 1),
  (1930.0, 0),
  (1040, 0),
  (150, 1),
  (1520.0, 0),
  (150, 1),
  (1180.0, 0),
  (1040, 0),
  (150, 1),
  (2270.0, 0),
  (150, 1),
  (430.0, 0),
  (50, 0)])

In [42]:
t_vec,3000-770-300

(array([   0.,  830., 1660., 2490., 3320.]), 1930)